In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [8]:
df_green = spark.read.parquet('data/pq/green/*/*')
df_green.createOrReplaceTempView('green')

In [10]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [12]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [2]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.createOrReplaceTempView('yellow')

C:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyspark\sql\dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [3]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [7]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [13]:
df_green_revenue = spark.read.parquet("data/report/revenue/green")
df_yellow_revenue = spark.read.parquet("data/report/revenue/yellow")

In [17]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [18]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=["hour", "zone"], how='outer')

In [19]:
df_join.show()

+-------------------+----+-----------------+--------------------+------------------+---------------------+
|               hour|zone|     green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+-----------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  13|             NULL|                NULL|1214.8000000000002|                   56|
|2020-01-01 00:00:00|  18|              7.8|                   1|               5.8|                    1|
|2020-01-01 00:00:00|  83|94.09999999999998|                   7|               9.8|                    1|
|2020-01-01 00:00:00| 114|             NULL|                NULL| 6256.430000000005|                  333|
|2020-01-01 00:00:00| 126|             NULL|                NULL|             170.6|                    2|
|2020-01-01 00:00:00| 134|            69.05|                   6|              NULL|                 NULL|
|2020-01-01 00:00:00| 161|           

In [20]:
df_join.write.parquet("data/report/revenue/total", mode='overwrite')

In [21]:
df_join = spark.read.parquet('data/report/revenue/total')
df_zones = spark.read.parquet('zones/')

In [23]:
df_result = df_zones.join(df_join, df_zones.LocationID == df_join.zone)

In [24]:
df_result = df_result.drop('LocationID', 'zone')

In [25]:
df_result.show()

+---------+------------+-------------------+-----------------+--------------------+------------------+---------------------+
|  Borough|service_zone|               hour|     green_amount|green_number_records|     yellow_amount|yellow_number_records|
+---------+------------+-------------------+-----------------+--------------------+------------------+---------------------+
|Manhattan| Yellow Zone|2020-01-01 00:00:00|             NULL|                NULL|106.99999999999999|                    6|
|Manhattan| Yellow Zone|2020-01-01 00:00:00|           107.52|                   6|6539.5100000000075|                  390|
|    Bronx|   Boro Zone|2020-01-01 00:00:00|             17.8|                   2|              31.0|                    1|
|    Bronx|   Boro Zone|2020-01-01 00:00:00|           160.04|                   6|57.620000000000005|                    2|
|    Bronx|   Boro Zone|2020-01-01 00:00:00|             11.8|                   1|106.52000000000001|                    4|


In [27]:
df_result.write.parquet('tmp/revenue-zones')